In [1]:
from utils import (load_data, CLASSIFICATION, REGRESSION, coffe_norm, get_model,
                  save_conf_matrix, error_angle, eval_model, get_generator_wrapper)
import numpy as np
import os
import matplotlib.pyplot as plt 
import sys
from glob import glob
import tensorflow as tf
from makiflow import set_main_gpu

set_main_gpu(1)

In [2]:
EXP_FOLDER = 'exp_classification_kfold_1'
BATCH_SIZE = 32
EXP_FOLDERS = glob(os.path.join(EXP_FOLDER, '*'))
EPOCHS = 30

In [ ]:
for path_to_exp in EXP_FOLDERS:
    # At start of each experiments - create new dataset (shuffle it)
    Xtrain, Ytrain, Xtest, Ytest, pred2params, config_data = load_data(
        path_exp_folder=path_to_exp,
        path_to_data='expand_double_modes',
        use_saved=True,
        size_hw=(336, 336),
        test_div=0.8,
        data_type=CLASSIFICATION,
        filename_config='config_data'
    )

    Ytrain = np.array(Ytrain).astype(np.int32)
    Ytest = np.array(Ytest).astype(np.int32)
    
    # Create model
    model, trainer, opt, global_step, sess = get_model(size_hw=(336, 336), batch_size=BATCH_SIZE, size_dataset=len(Ytrain))
    gen = get_generator_wrapper(Xtrain, Ytrain, BATCH_SIZE)
    accur_list = []
    max_acc_np = np.float32(0.0)

    for epoch_num in range(EPOCHS):
        # Each epoch:
        # Train -> eval -> create folder -> save weights/arch -> save results
        print('ep: ', epoch_num)
        info1 = trainer.fit_generator(
            gen, optimizer=opt, epochs=1, 
            iter=len(Xtrain)//(2*BATCH_SIZE), print_period=150, global_step=global_step
        )
        acc_test, predictions = eval_model(model, Xtest, Ytest, BATCH_SIZE, calc_mean=False)
        acc_float = np.mean(acc_test)
        max_acc_np = np.max([max_acc_np, acc_float])
        print('acc test: ', acc_float)
        accur_list.append(acc_float)
        
        # Create folder
        path_to_s_epoch = os.path.join(path_to_exp, f'acc_{round(acc_float, 2)}_ep_{epoch_num}_exp')
        os.makedirs(path_to_s_epoch, exist_ok=True)
        # Save weights/arch
        model.save_weights(os.path.join(path_to_s_epoch, 'weights.ckpt'))
        model.save_architecture(os.path.join(path_to_s_epoch, 'model.json'))
        # Save results
        save_conf_matrix(
            Ytest, predictions, 
            os.path.join(path_to_s_epoch, 'conf_matrix'),
            save_np=True
        )
        error_angle(
            os.path.join(path_to_s_epoch, 'error_angle_all_class'), 
            config_data, Ytest, predictions,
            save_np=True
        )
        error_angle(
            os.path.join(path_to_s_epoch, 'error_angle_20_19_classes'), 
            config_data, Ytest, predictions, class_find=[20, 19],
            save_np=True
        )
        error_angle(
            os.path.join(path_to_s_epoch, 'error_angle_20_classes'), 
            config_data, Ytest, predictions, class_find=[20],
            save_np=True
        )
        error_angle(
            os.path.join(path_to_s_epoch, 'error_angle_19_classes'), 
            config_data, Ytest, predictions, class_find=[19],
            save_np=True
        )
    np.save(os.path.join(path_to_exp, 'max_acc.npy'), np.float32(max_acc_np))
    fig = plt.figure(figsize=(8, 8))
    plt.plot(accur_list)
    fig.savefig(os.path.join(path_to_exp, 'acc.png'))
    # Clear plt figures
    plt.close('all')
    # close session and reset graph
    sess.close()
    tf.reset_default_graph()

100%|██████████| 3780/3780 [00:07<00:00, 526.76it/s]

train :  34020
test:  3780







Loss is built.


ep:  0



  0%|          | 0/531 [00:00<?, ?it/s]

New optimizer is used.


...

 28%|██▊       | 150/531 [01:00<02:39,  2.39it/s]

Epoch: 0
Cross_Entropy: 0.3854159 
Training_Loss: 0.3854159 




 57%|█████▋    | 301/531 [02:02<01:27,  2.63it/s]

Epoch: 0
Cross_Entropy: 0.3168537 
Training_Loss: 0.3168537 




 70%|██████▉   | 371/531 [02:30<00:53,  3.00it/s]

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(Xtest[-100])

In [ ]:
Ytest[-100]

In [ ]:
len(acc_test)

In [ ]:
len(predictions)

In [ ]:
1

In [ ]:
2